In [147]:
import numpy as np
from sympy import *
from sympy.matrices import Matrix

Define Testdata

In [148]:
test_pos_x = 2.3029
test_pos_y = 0
test_pos_z = 1.9466

test_roll  = 0
test_yaw   = 0
test_pitch = 0

First create the symbols

In [149]:
# Create symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') #theta_i
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

Then create modified DH parameters

In [150]:
### KUKA KR210 ###
# Create Modified DH parameters
s = {alpha0:     0,  a0:      0, d1:  0.75,  
     alpha1: -pi/2,  a1:   0.35, d2:     0, q2: q2-pi/2, 
     alpha2:     0,  a2:   1.25, d3:     0,       
     alpha3: -pi/2,  a3: -0.054, d4:  1.50,
     alpha4:  pi/2,  a4:      0, d5:     0,
     alpha5: -pi/2,  a5:      0, d6:     0,
     alpha6:     0,  a6:      0, d7: 0.303, q7: 0}


Define the transformation matrix

In [151]:
# Define Modified DH Transformation matrix
def build_transformation_matrix( theta, alpha, a, d ):
    T = Matrix([[            cos(theta),           -sin(theta),           0,             a ],
                [ sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d ],
                [ sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  cos(alpha)*d ],
                [                     0,                     0,           0,             1 ]]) 
    T = T.subs(s)
       
    return(T)

# Modified DH params
T0_1 = build_transformation_matrix( q1, alpha0, a0, d1 )
T1_2 = build_transformation_matrix( q2, alpha1, a1, d2 )
T2_3 = build_transformation_matrix( q3, alpha2, a2, d3 )
T3_4 = build_transformation_matrix( q4, alpha3, a3, d4 )
T5_5 = build_transformation_matrix( q5, alpha5, a5, d5 )
T5_6 = build_transformation_matrix( q6, alpha5, a5, d6 )
T6_G = build_transformation_matrix( q7, alpha6, a6, d7 )


Correct the Orientation difference between the definition of the gripper_link in the urdf-file and the Denavit-Hardenberg Convention.

First apply a body-fixed rotation about the z-axis followed by a body-fixed rotation about the y-axis.

R_corr is the composition of those two rotations.

In [152]:
#Correct the Gripper
R_z =  Matrix([[     cos(np.pi), -sin(np.pi),             0, 0],
               [     sin(np.pi),  cos(np.pi),             0, 0],
               [              0,           0,             1, 0],
               [              0,           0,             0, 1]])

R_y =  Matrix([[  cos(-np.pi/2),           0, sin(-np.pi/2), 0],
               [              0,           1,             0, 0],
               [ -sin(-np.pi/2),           0, cos(-np.pi/2), 0],
               [              0,           0,             0, 1]])

R_corr = simplify(R_z * R_y)

Create individual transformation matrices

In [153]:
# Create individual transformation matrices
T0_2 = simplify(T0_1 * T1_2) # base_link to link_2
T0_3 = simplify(T0_2 * T2_3) # base_link to link_3
T0_4 = simplify(T0_3 * T3_4) # base_link to link_4
T0_5 = simplify(T0_4 * T4_5) # base_link to link_5
T0_6 = simplify(T0_5 * T5_6) # base_link to link_6
T0_G = simplify(T0_6 * T6_G) # base_link to gripper_link
T0_G_corr = simplify(T0_G * R_corr) # Total HT between base_link and gripper_link with orientation correction applied

#T0_G = simplify(T0_1 * T1_2 * T2_3 * T3_4 * T4_5 * T5_6 * T6_G)
sub_dict = { q1: -2.77, q2: 0.36, q3: -2.41, q4: 4.56, q5: 1.31, q6: 4.70 }
sub_dict_zero = { q1: 0, q2: 0, q3: 0, q4: 0, q5: 0, q6: 0 }
# Evaluate transforms
print ("T0_1 = ", T0_1.evalf(subs=sub_dict_zero),"\n")
print ("T0_2 = ", T0_2.evalf(subs=sub_dict_zero),"\n")
print ("T0_3 = ", T0_3.evalf(subs=sub_dict_zero),"\n")
print ("T0_4 = ", T0_4.evalf(subs=sub_dict_zero),"\n")
print ("T0_5 = ", T0_5.evalf(subs=sub_dict_zero),"\n")
print ("T0_6 = ", T0_6.evalf(subs=sub_dict_zero),"\n")
print ("T0_G = ", T0_G.evalf(subs=sub_dict_zero),"\n")
print  ("T0_G_corr = ", T0_G_corr.evalf(subs=sub_dict_zero),"\n")
#print ("T0_1 = ", T0_1,"\n")
#print ("T0_2 = ", T0_2,"\n")
#print ("T0_3 = ", T0_3,"\n")
#print ("T0_4 = ", T0_4,"\n")
#print ("T0_5 = ", T0_5,"\n")
#print ("T0_6 = ", T0_6,"\n")
#print ("T0_G = ", T0_G,"\n")
#print  ("T0_G_corr = ", T0_G_corr,"\n")

T0_1 =  Matrix([[1.00000000000000, 0, 0, 0], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 0.750000000000000], [0, 0, 0, 1.00000000000000]]) 

T0_2 =  Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0, 0.75],
[  0,   0,   0,  1.0]]) 

T0_3 =  Matrix([
[  0, 1.0,   0, 0.35],
[  0,   0, 1.0,    0],
[1.0,   0,   0,  2.0],
[  0,   0,   0,  1.0]]) 

T0_4 =  Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T0_5 =  Matrix([
[  0, 1.0,   0,  1.85],
[  0,   0, 1.0,     0],
[1.0,   0,   0, 1.946],
[  0,   0,   0,   1.0]]) 

T0_6 =  Matrix([
[  0,    0, 1.0,  1.85],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T0_G =  Matrix([
[  0,    0, 1.0, 2.153],
[  0, -1.0,   0,     0],
[1.0,    0,   0, 1.946],
[  0,    0,   0,   1.0]]) 

T0_G_corr =  Matrix([
[                  1.0,                     0, 6.12323399573677e-17, 2.153],
[-7.49879891330929e-33,                   1.0, 1.2

Extract End-Effector position and orient from request
Calculate joint angles using Geometric IK method

In [154]:
# Build a rotation matrix from the Roll, Pitch and Yaw angles
def get_wrist_rot_matrix(roll, pitch, yaw):
        
    # Rrpy = Rx * Ry * Rz
    Rrpy = Matrix([[    cos(yaw)*cos(pitch),   cos(yaw)*sin(pitch)*sin(roll) - sin(yaw)*cos(roll),    cos(yaw)*sin(pitch)*cos(roll) + sin(yaw)*sin(roll)],
                   [    sin(yaw)*cos(pitch),   sin(yaw)*sin(pitch)*sin(roll) + cos(yaw)*cos(roll),    sin(yaw)*sin(pitch)*cos(roll) - cos(yaw)*sin(roll)],
                   [            -sin(pitch),             cos(pitch)*sin(roll),                                       cos(pitch)*cos(roll)               ]])

    print("Rrpy", Rrpy)
    return Rrpy

Find the location of the WC relative to the base frame

In [155]:
def get_wrist_pos(px, py, pz, Rrpy):
           
    end_effector_length = 0.453
    
    
    lx = Rrpy[ 0, 0 ]
    ly = Rrpy[ 1, 0 ]
    lz = Rrpy[ 2, 0 ]

    print("lx:",lx," ly:",ly," lz:",lz)

    d6 = 0

    # Calculate Wrist Center
    wx = px - ( end_effector_length + d6 ) * lx
    wy = py - ( end_effector_length + d6 ) * ly
    wz = pz - ( end_effector_length + d6 ) * lz
    
    # Modify Rrpy to take in transformation from gripper
    Rrpy = Rrpy.row_join(Matrix([[px],[py],[pz]]))
    Rrpy = Rrpy.col_join(Matrix([[0,0,0,1]]))

    print("wx:",wx," wy:",wy," wz:",wz)
    print("Rrpy:", Rrpy)
    return wx, wy, wz, Rrpy


In [156]:
Rrpy = get_wrist_rot_matrix(test_roll, test_pitch, test_yaw)
wx, wy, wz, Rrpy = get_wrist_pos(test_pos_x, test_pos_y, test_pos_z, Rrpy)

test_wx = wx
test_wy = wy
test_wz = wz
#test_wx = 1.8499
#test_wy = 0
#test_wz = 1.9466

Rrpy Matrix([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
lx: 1  ly: 0  lz: 0
wx: 1.84990000000000  wy: 0  wz: 1.94660000000000
Rrpy: Matrix([[1, 0, 0, 2.30290000000000], [0, 1, 0, 0], [0, 0, 1, 1.94660000000000], [0, 0, 0, 1]])


find joint variables, q1, q2 and q3, such that the WC has coordinates equal to equation (3). This is the hard step. One way to attack the problem is by repeatedly projecting links onto planes and using trigonometry to solve for joint angles. Unfortunately, there is no generic recipe that works for all manipulators so you will have to experiment.

In [157]:
# Calculate joint angles using Geometric IK method
def get_theta_123 (wx, wy, wz):
    
    theta1 = atan2(wy, wx)
    a_1 = 0.35
    a_2 = 1.25
    d_1 = 0.75
    d_4 = 1.50
    a_3 = -0.054
    
    
    wc_dist_x = sqrt(wx**2 + wy**2) - a_1 # Subtract a1 as horizontal offset from robot-base
    wc_dist_z = wz - d_1 # Subtract d1 as vertical offset from robot base
    
    x_c = sqrt( wc_dist_x**2 + wc_dist_z**2 )
    y_c = wc_dist_z

    L_25 = sqrt(x_c**2 + y_c**2)
    L_35 = sqrt(a_3**2+d_4**2)

    beta_4 = atan2(a_3,d_4)
    cos_beta_3 = (( -L_25**2 + a_2**2 + L_35**2) / ( 2 * a_2 * L_35 ))
    beta_3 = atan2( cos_beta_3, sqrt( 1 - cos_beta_3**2 ))
    theta3 = np.pi/2 - beta_3 - beta_4
    
    beta_2 = atan2( y_c, x_c )
    cos_beta_1 = ( ( -L_35**2 + L_25**2 + a_2**2 ) / ( 2 * L_25 * a_2 ) )
    beta_1 = atan2(  cos_beta_1, sqrt( 1 - cos_beta_1**2 ) ) 
    theta2 = np.pi/2 - beta_1 - beta_2

    print("theta1,2,3:",theta1, theta2, theta3)
        
    return theta1.evalf(), theta2, theta3

theta1, theta2, theta3 = get_theta_123( test_wx, test_wy, test_wz )

theta1,2,3: 0 0.114689449584584 1.95998482362893


once the first three joint variables are known, calculate 03R via application of homogeneous transforms up to the WC.

In [158]:
def transform_to_wc(theta1, theta2, theta3, Rrpy):
 
    R0_3 = simplify( T0_1 * T1_2 * T2_3 )
    #R0_3 = Matrix([[    sin(theta2 + theta3)*cos(theta1),   cos(theta1)*cos(theta2 + theta3),   -sin(theta1)],
                   #[   sin(theta1)*sin(theta2 + theta3),   sin(theta1)*cos(theta2 + theta3),   cos(theta1)],
                   #[   cos(theta2 + theta3),           -sin(theta2 + theta3),          0]])
    print("R0_3:", R0_3.evalf(subs={q1: theta1, q2: theta2, q3: theta3}))
    R0_3 = R0_3.evalf(subs={q1: theta1, q2: theta2, q3: theta3})
    R3_6_eval = R0_3.inv() * Rrpy
    R3_6_var = simplify( T3_4 * T4_5 * T5_6 )
    print("R3_6:", R3_6_var)
    return R3_6_var, R3_6_eval
        
R3_6_var, R3_6_eval = transform_to_wc( theta1, theta2, theta3, Rrpy )

R0_3: Matrix([
[ 0.875716781259277, -0.482825143318875,   0, 0.493047729628134],
[                 0,                  0, 1.0,                 0],
[-0.482825143318875, -0.875716781259277,   0,   1.9917879638039],
[                 0,                  0,   0,               1.0]])
R3_6: Matrix([
[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4), -0.054],
[                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5),    1.5],
[-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5),      0],
[                                         0,                                          0,                0,      1]])


find a set of Euler angles corresponding to the rotation matrix

In [159]:
def get_theta_456(R3_6_var, R3_6_eval):
    r13 = R3_6_eval[0,2]
    r33 = R3_6_eval[2,2]
    r23 = R3_6_eval[1,2]
    r22 = R3_6_eval[1,1]
    r21 = R3_6_eval[1,0]
    
    print("r13: ", R3_6_var[0,2])
    print("r23: ", R3_6_var[1,2])
    print("r33: ", R3_6_var[2,2])    
    sin_q5_var = sqrt(R3_6_var[0,2]**2 + R3_6_var[2,2]**2)
    print("sin(q5): ", sin_q5_var)
    # sin(q5)² * ( cos(q4)² + sin(q4)²)
    # cos(q4)² + sin(q4)² = 1
    # => sqrt(r13² + r33²) = sin(q5)
    sin_q5 = sqrt(r13**2 + r33**2).evalf()
           
    theta5 = atan2( sin_q5, r23 ).evalf()
    
    if( sin_q5 < 0 ):
        theta4 = atan2(-r33,  r13).evalf()
        theta6 = atan2( r22, -r21).evalf()
    else:
        theta4 = atan2( r33,  -r13).evalf()
        theta6 = atan2( -r22, r21).evalf()
    
    print("theta4, theta5, theta6: ",theta4, theta5, theta6)

    
    return theta4, theta5, theta6

theta4, theta5, theta6 = get_theta_456( R3_6_var, R3_6_eval )

r13:  -sin(q5)*cos(q4)
r23:  cos(q5)
r33:  sin(q4)*sin(q5)
sin(q5):  sqrt(sin(q4)**2*sin(q5)**2 + sin(q5)**2*cos(q4)**2)
theta4, theta5, theta6:  0 2.63771470717117 3.14159265358979


Test your IK based on your FK

In [168]:
print("px, py, pz:   ", test_pos_x, test_pos_y, test_pos_z)
print("r , p , y :   ", test_roll, test_pitch, test_yaw)
print("wrist center: ", wx, wy, wz)
print("joint angles:")
print("theta 1:", theta1)
print("theta 2:", theta2)
print("theta 3:", theta3)
print("theta 4:", theta4)
print("theta 5:", theta5)
print("theta 6:", theta6,"\n")


theta_values = { q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6 }
print("R0_6: ", T0_G_corr.evalf(subs=theta_values))
print("Rrpy: ", Rrpy.evalf())

px, py, pz:    2.3029 0 1.9466
r , p , y :    0 0 0
wrist center:  1.84990000000000 0 1.94660000000000
joint angles:
theta 1: 0
theta 2: 0.114689449584584
theta 3: 1.95998482362893
theta 4: 0
theta 5: 2.63771470717117
theta 6: 3.14159265358979 

R0_6:  Matrix([
[-1.33907057366955e-16, -2.44929359829471e-16,                   1.0, -0.278478691538179],
[ 1.49975978266186e-32,                  -1.0, -2.44929359829471e-16,                  0],
[                  1.0, -1.78001720109179e-32,  1.33907057366955e-16,    1.0072853496542],
[                    0,                     0,                     0,                1.0]])
Rrpy:  Matrix([[1.00000000000000, 0, 0, 2.30290000000000], [0, 1.00000000000000, 0, 0], [0, 0, 1.00000000000000, 1.94660000000000], [0, 0, 0, 1.00000000000000]])
